In [ ]:
try:
    assert api_key
except NameError:
    api_key = !op read "op://Private/Google AI Studio API key/credential"
    assert api_key
    api_key = api_key[0]

In [ ]:
import requests
url = f"https://generativelanguage.googleapis.com/v1beta/models?key={api_key}"
models = requests.get(url).json()["models"]
models

In [ ]:
[f"{x['name']} {prop} {x[prop]}" for x in models for prop in ["inputTokenLimit","outputTokenLimit"] ]

In [ ]:
!pip install -q google-api-python-client google-generativeai

In [ ]:
import googleapiclient
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import mimetypes

In [ ]:
GENAI_DISCOVERY_URL = f"https://generativelanguage.googleapis.com/$discovery/rest?version=v1beta&key={api_key}"
# Initialize Google API Client
discovery_docs = requests.get(GENAI_DISCOVERY_URL)
genai_service = googleapiclient.discovery.build_from_document(
    discovery_docs.content, developerKey=api_key)

In [ ]:
mimetypes.guess_type("abc.webp")

In [ ]:
# Prepare file to upload to GenAI File API
file_path = "frieren.png"

media = MediaFileUpload(file_path, mimetype=mimetypes.guess_type(file_path)[0])
body = {"file": {"displayName": "Frieren meme"}}

# Upload file
create_file_request = genai_service.media().upload(media_body=media, body=body)
create_file_response = create_file_request.execute()
file_uri = create_file_response["file"]["uri"]
file_mimetype = create_file_response["file"]["mimeType"]
print("Uploaded file: " + file_uri)

In [ ]:
# Make Gemini 1.5 API LLM call
prompt = "Describe the image with a creative description"
model = "models/gemini-1.5-pro-latest"
contents = {"contents": [{ 
  "parts":[
    {"text": prompt},
    {"file_data": {"file_uri": file_uri, "mime_type": file_mimetype}}]
}]}
genai_request = genai_service.models().generateContent(model=model, body=contents)
resp = genai_request.execute()
print(len(resp["candidates"]))

In [ ]:
from IPython.display import display,Image
from pprint import pprint
display(Image(file_path, width=400))
pprint(resp["candidates"][0]["content"]["parts"][0]["text"])

In [ ]:
# Prepare file to upload to GenAI File API
file_path = "beans.txt"

# media = MediaFileUpload(file_path, mimetype="application/json")
media = MediaFileUpload(file_path, mimetype=mimetypes.guess_type(file_path)[0])
body = {"file": {"displayName": "A text file"}}

# Upload file
create_file_request = genai_service.media().upload(media_body=media, body=body)
create_file_response = create_file_request.execute()
file_uri = create_file_response["file"]["uri"]
file_mimetype = create_file_response["file"]["mimeType"]
print("Uploaded file: " + file_uri)

In [ ]:
# Make Gemini 1.5 API LLM call
prompt = "Describe the contents of the text file"
model = "models/gemini-1.5-pro-latest"
contents = {"contents": [{ 
  "parts":[
    {"text": prompt},
    {"file_data": {"file_uri": file_uri, "mime_type": file_mimetype}}]
}]}
genai_request = genai_service.models().generateContent(model=model, body=contents)
resp = genai_request.execute()
print(len(resp["candidates"]))

In [ ]:
# Prepare file to upload to GenAI File API
file_path = "explore.ipynb"

# media = MediaFileUpload(file_path, mimetype="application/json")
media = MediaFileUpload(file_path, mimetype="text/plain")
body = {"file": {"displayName": "This jupyter notebook"}}

# Upload file
create_file_request = genai_service.media().upload(media_body=media, body=body)
create_file_response = create_file_request.execute()
file_uri = create_file_response["file"]["uri"]
file_mimetype = create_file_response["file"]["mimeType"]
print("Uploaded file: " + file_uri)

In [ ]:
file_mimetype

In [ ]:
import json
this_notebook = open("explore.ipynb", "r").read()

# Make Gemini 1.5 API LLM call
prompt = """I'm just getting started with the Gemini 1.5 API. I know that uploading files and caching the url is going to be an important part of the workflow.
Perhaps a thing like file_path_to_remote_url_mapping = {} with a helper method to get-or-create entry will be useful.
The current goal is to streamline the file upload process in a way that I can just give it a file path and it will cache the remote url.
Then, I can create a prompt and evaluate it against any of the cached files.
We can use fzf to select the file to evaluate against.
The file uploading and inference API calls are python components, but the fzf call will be a pwsh invocation.
So, we need to create a .py file we can invoke with pwsh to upload a file and get the url, 
then in pwsh we can use a json file to act as a simple dictionary to cache the file urls
lets imagine the gemini.ps1 file to output something like

Welcome to the Gemini 1.5 API
Found 23 files. (read from files.json)
Found 13 prompts. (read from prompts/*.txt)

1. Upload a file
2. Create a prompt
3. Execute a prompt

Choice: 1
Enter the file path: explore.ipynb
"explore.ipynb" already exists, overwrite? (y/n): y
Uploaded file and saved the url to the cache.

Found 24 files. (read from files.json)
Found 13 prompts. (read from prompts/*.txt)

1. Upload a file
2. Create a prompt
3. Execute a prompt

Choice: 2

Enter a name for the prompt: "Describe the jupyter notebook"
(here, the pwsh file calls `hx prompts/${name}.txt` to open the file in the helix CLI editor, blocking until the user finishes)

Found 24 files. (read from files.json)
Found 14 prompts. (read from prompts/*.txt)

1. Upload a file
2. Create a prompt
3. Execute a prompt

Choice: 3

(here, the pwsh file does something like
$chosen_file_name = $cached_file_names | fzf
$file_url = $cached_file_urls[$chosen_file_name]
try {
Push-Location prompts
$chosen_prompt_name = $cached_prompt_names | fzf
} finally {
Pop-Location
}
$prompt = Get-Content prompts\$chosen_prompt_name
payload = [pscustomobject]@{
  prompt = $prompt
  file_url = $file_url
}
python gemini_invoke.py $($payload | ConvertTo-Json)

Please, using markdown syntax with code blocks, write the code for the following files:
- gemini.ps1
- gemini_file_upload.py
- gemini_invoke.py

Let's think step by step.
For each file, write an English requirements paragraph that describes the decision flows and expectations of the connected components.
After writing the requirements for the individual files, review the interdependencies for any requirements that are not yet clear.

Finally, for each file, output the code similar to the following

```gemini.ps1
%%%CODE%%%
```

and replace `%%%CODE%%%` with the code that fulfills the requirements.

Finally, here is the content of this jupyter notebook
===
""" + this_notebook
print(len(prompt))

In [ ]:
assert len(prompt) < 100_000, "Make sure to clear cell outputs containing images"

In [ ]:
model = "models/gemini-1.5-pro-latest"
contents = {
    "contents": [
        {
            "parts": [
                {"text": prompt},
                # {"file_data": {"file_uri": file_uri, "mime_type": file_mimetype}}
            ]
        }
    ]
}
genai_request = genai_service.models().generateContent(model=model, body=contents)
resp = genai_request.execute()
print(resp)

In [ ]:
print(len(resp["candidates"]))
pprint(resp["candidates"][0]["content"]["parts"][0]["text"])

In [ ]:
with open("reply.md","w") as f:
    f.write(resp["candidates"][0]["content"]["parts"][0]["text"])